# LLM From scratch

In [1]:
# import
import torch
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

## I. Text Small Analysis

In [3]:
# Open The text file
with open("../input.txt","r", encoding = "utf-8") as f:
    text = f.read()
print(text[:100])
print(f'the text has a length of {len(text)} characters')

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
the text has a length of 1115394 characters


## II. Tokenizer and Encoder/Decoder

Tokenize : Transform Text --> Integers

Short size of vocabulary but Long sequence length : Character-level Tokenizer
Simple Tokeinizer : with a mapping of characters to integers using the position in chars (The one we will use)

Long size of vocabulary but Short sequence length : sub-word/Word-level Tokenizer
Google Tokenizer : SentencePiece : sub-words units level tokenizer
OPENAI : Tiktoken : BPE Tokenizer


In [7]:
# Characters of the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Encoder : Transform text --> Vectors/integers $\newline$
Decoder : Transform Vectors/integers --> text

In [11]:
# Create the mapping from characters to integer
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
def encode(s): return [stoi[c] for c in s]
def decode(L): return "".join([itos[i] for i in L])

print(encode("je suis Osiris"))
print(decode([48, 43, 1, 57, 59, 47, 57, 1, 27, 57, 47, 56, 47, 57]))

[48, 43, 1, 57, 59, 47, 57, 1, 27, 57, 47, 56, 47, 57]
je suis Osiris


## Using **PyTorch Tensors** to store text as **embeddings**

- **Ease of use**: All modules (**nn.Embedding**, **nn.Linear**, **Transformer**, etc.) expect **torch.Tensor**.  

- **GPU/TPU**: Once it’s a **tensor**, it can be moved to **GPU/TPU** with `x.to("cuda")` or `x.to("tpu")`. **NumPy** only works on **CPU**.  

- **Backward / Autograd**: **PyTorch tensors** keep track of operations for **automatic gradient computation**.  

  **Example**:  
  ```python
  import torch

  x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
  y = x * 2
  y.backward(torch.ones_like(x))
  print(x.grad)  # x remembers its expression with respect to y allowing it to compute x.grad automatically

    ```
- `x` remembers its expression with respect to `y`, which allows computing **x.grad** automatically.  

- **Integrated tensors** in **PyTorch** (with **optimizers** like **Adam**, **SGD**, **loss functions**, training on **mini-batches**).  

- **PyTorch** is optimized in **C++/CUDA** → ultra-fast **matrix computations**.  


In [14]:
data = torch.tensor(encode(text))
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [16]:
# Spliting train and test
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

## III. Chunking


**Chunk = split the text** into smaller pieces (e.g. paragraphs of **300–500 tokens**).

### 🔹 Why chunk?
- **Context window limit**: LLMs can only read a fixed number of tokens (2k, 4k, 32k...).$\newline$
--> Chunking helps fit large texts into the model's context window.$\newline$
--> Attention mechanisms in transformers have quadratic complexity with respect to input length, so shorter inputs are more efficient.$\newline$
- **Memory efficiency**: smaller chunks use less memory during processing.
- **Better local coherence**: smaller chunks keep the meaning clear.  
- **Efficient search (RAG)**: embeddings are stored per chunk, making retrieval precise.  
- **Performance**: faster similarity search + less dilution in attention.  

Rule of thumb: chunks of **200–500 tokens** usually work best.

In [17]:
# Exemple
block_size = 8
x = train_data[:block_size] # input of the transformer : first block size characters
y = train_data[1:block_size+1] # next block size character offset by one --> target for each pos
for t in range(block_size):
    context = x[:t+1] # all the tokens before the token at t
    target = y[t]     # expected token at t
    print(f'input: {context}, target: {y[t]}')


input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57, 58])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57, 58,  1])
input: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ff185012100>>, context: tensor([18, 47, 56, 57, 58,  1, 15])
input: <bound method Kernel.raw_input of 

## Batching

- **Batch** = a set of sequences processed in parallel on the GPU.  
- **Main purpose**: speed up training with GPU parallelization.  
- Each sequence in the batch is **independent**:  
  - The loss is calculated for each sequence, then combined to update the model weights.  
- Batching is used **only to process multiple sequences in parallel** for GPU efficiency, **without sharing context between them**.
- **Weight update**: after computing the losses for all the sequences in the batch, **the gradients are combined to update the model’s weights.**


In [26]:

batch_size = 4  # Number of sequences (sentences) processed in parallel
block_size = 8  # Maximum length of a sequence in tokens (model context window)

def get_batch(split):
    # Generate small batch of data of inputs x and target y
    data = train_data if split =='train' else val_data
    # Choice of random offset (starting point) during training --> Avoid overfitting (<== if we start always at the same point)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])       # context (stacked by row in a batch_size x block_size Tensor)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])   # target/prediction
    return x,y

# Visualisation
xb,yb = get_batch('train')
print('-------------Inputs-------------')
print(xb.shape)
print(xb)
print('-------------Targets-------------')
print(yb.shape)
print(yb)
print('---------------------------------')

-------------Inputs-------------
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
-------------Targets-------------
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------------------


In [27]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"input = {context.tolist()}, target = {target}")

input = [24], target = 43
input = [24, 43], target = 58
input = [24, 43, 58], target = 5
input = [24, 43, 58, 5], target = 57
input = [24, 43, 58, 5, 57], target = 1
input = [24, 43, 58, 5, 57, 1], target = 46
input = [24, 43, 58, 5, 57, 1, 46], target = 43
input = [24, 43, 58, 5, 57, 1, 46, 43], target = 39
input = [44], target = 53
input = [44, 53], target = 56
input = [44, 53, 56], target = 1
input = [44, 53, 56, 1], target = 58
input = [44, 53, 56, 1, 58], target = 46
input = [44, 53, 56, 1, 58, 46], target = 39
input = [44, 53, 56, 1, 58, 46, 39], target = 58
input = [44, 53, 56, 1, 58, 46, 39, 58], target = 1
input = [52], target = 58
input = [52, 58], target = 1
input = [52, 58, 1], target = 58
input = [52, 58, 1, 58], target = 46
input = [52, 58, 1, 58, 46], target = 39
input = [52, 58, 1, 58, 46, 39], target = 58
input = [52, 58, 1, 58, 46, 39, 58], target = 1
input = [52, 58, 1, 58, 46, 39, 58, 1], target = 46
input = [25], target = 17
input = [25, 17], target = 27
input = [2

## IV. Language Models

## Bigram Model : simplest Language Model

- A **bigram model** is a very simple language model that predicts a token based **only on the previous token**.  
- **Unigram model** → assumes each token is independent.  
- **Bigram model** → probability of a token depends only on the one before it.  
- **N-gram model** → generalization where the token depends on the previous *n-1* tokens.  


### Formula

For a sequence of tokens $ w_1, w_2, \dots, w_n $:  

$$
P(w_1, w_2, \dots, w_n) \approx \prod_{i=2}^n P(w_i \mid w_{i-1})
$$


##  Reminder: Role of `nn.Embedding` in PyTorch

- **`nn.Embedding(num_embeddings, embedding_dim)`** = a **lookup table** (similar to dictionnary) that maps token indices to vectors.  

- Each token (represented by an **integer index**) is associated with a **learnable vector**.  

- In a **language model**:
  - Input: token indices (`idx`).
  - Output: embeddings (vectors) that represent each token.  

- Example with `nn.Embedding(vocab_size, vocab_size)` in a Bigram model:
  - Each token index is mapped to a **vector of size `vocab_size`**.
  - This vector acts as the **logits** (scores) for predicting the **next token**.  
  - Applying **softmax** on these logits gives a probability distribution over the vocabulary.  

➡️ In short: **`nn.Embedding` converts discrete tokens into continuous vectors** that the model can use for learning and prediction.


### 1. Loss : Evaluating the quality of the model's predictions

In [47]:
class BigramLangaugeModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # Each token is associated to a vocab_size dimension vector 
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx, targets = None):
        
        # dim of idx and target = (B,T)
        logits = self.token_embedding_table(idx) # size (B,T,C)
        # Computation of the loss
        if targets is None:
            loss = None
        else:
            #A. reshapping tensors to adapt them to Pytorch.Cross Entropy input size requirement
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            #B.Computation of the Loss
            loss = F.cross_entropy(logits, targets)

        return loss, logits
    
    def generate(self,idx,max_new_tokens):
        # idx = (B,T) array of indices in the current context
        # Goal : 
        for _ in range(max_new_tokens):
            # get predictions
            loss, logits = self(idx) # loss = null because no targets
            # Bigram --> Focusing only on the last token
            logits = logits[:,-1,:] # size (B,C) (a plane in the cubic matrix)
            # Getting probabilities with softmax
            probs = F.softmax(logits,dim = -1)
            # sample from the distribution
            idx_next = torch.multinomial(probs,num_samples=1) # size = (B,1) = the prediction for what come next for each line
            # Append it to the running sequence
            idx = torch.cat((idx,idx_next), dim=1) #(B,T+1) we add the last token
        return idx


In [48]:
m = BigramLangaugeModel(vocab_size)
# Calling the model directly (m(xb, yb)) automatically invokes forward with PyTorch hooks
# This ensures proper handling of autograd, hooks, and training/evaluation modes
loss, logits  = m(xb,yb)
print(logits.shape) # size B = batch_size, T = block_size, C = vocab_size
print(loss) # around -ln(1/65)

torch.Size([32, 65])
tensor(4.4399, grad_fn=<NllLossBackward0>)


In [57]:
# Testing the first llm
print(decode(m.generate(idx = torch.zeros(1,1, dtype = torch.long), max_new_tokens = 100)[0].tolist()))


?ACVWq-AJAhMrXfLgNVTNySlxdRkdbm
XL
qM?RbO!PH PUWHVW3aYaepXWYxhnGCbgNzm;WSYtw?mzKSi:-:y!&'3voqcm$cISC


##### 1st Conclusion:
- the prediction doesn't take the context into account : it only predicts the next token based on the previous one, ignoring the context of the entire sequence.

- The generated text is not deterministic: the next token is sampled randomly according to the probability distribution, so it can be incoherent and change with each generation.

## Training the Bigram Model 
To make it less random



In [59]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr = 1e-3)

In [60]:
batch_size = 32
for steps in range(20000):
    # Sample a batch of data
    xb,yb = get_batch('train')

    # Evaluate the loss
    loss, logits = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.514085531234741


In [63]:
print(decode(m.generate(idx = torch.zeros(1,1, dtype = torch.long), max_new_tokens = 100)[0].tolist()))


Whablo w.
DYou thouivif s he y anere tla her.
Y keslos, celd d uick bepat fourerubefo; lds Prd.
Whiv


##### Conclusion : 
- Better than random (better structure, less hazardous character succession) but still not great
- The bigram model captures some local structure but lacks long-range context.

This is the simplest possible model, but it is very limited in its ability to generate coherent text.

## Self-attention Introduction

Self-attention is a mechanism that allows a model to weigh the importance of different positions in a sequence when encoding information. Instead of processing tokens independently, each token can "look at" all others and decide which ones are most relevant.

- **Query (Q):** current token’s perspective - what a token looks for  
- **Key (K):** content of each token - what a token offers  
- **Value (V):** actual information carried by each token - what will be communicated

The attention score is computed as the **dot product between Q and K**, scaled by the dimension of the keys, and passed through a softmax. These scores weight the Values (V), producing the output representation.  

$$
\text{Attention}(Q, K, V) = \text{softmax}\!\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

This mechanism captures long-range dependencies and relationships between tokens efficiently.


### Multi-Head Attention (Heads)

A **head** is one set of learned projections for Queries, Keys, and Values.  
Each head maps the input embeddings into a lower-dimensional space (`head_size`) and computes attention there.  
Each head acts as a **latent subspace** where token relationships are evaluated.  

- All heads perform the **same attention operation**, but with **different learned weights** (`Wq`, `Wk`, `Wv`).
- This allows each head to capture **different types of relationships**:
  - one head may focus on short-range syntax,
  - another on long-range dependencies,
  - another on semantic similarity.

By combining multiple heads (concatenating their outputs), the model gains a richer, multi-perspective understanding of the sequence.  

👉 **Difference between heads**: *not the computation itself, but their independent learned parameters.*



### Self-Attention vs Cross-Attention

**Self-Attention**  
- Each token attends to **other tokens in the same sequence**.  
- Queries, Keys, and Values all come from the **same input**.  
- Captures **internal dependencies** within a sequence.  

**Cross-Attention**  
- Each token attends to a **different sequence** (e.g., decoder attends to encoder outputs).  
- Queries come from one sequence, Keys and Values come from another.  
- Captures **relationships between two sequences**, e.g., aligning text to context or translation.  

💡 Intuition:  
- Self-attention = “tokens talking to themselves.”  
- Cross-attention = “tokens talking to another sequence.”


In [3]:
# Exemple 
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [22]:
# We want x[b,t] = mean_{i=t} x[b,i]
# The simplest way for a token to know all the previous context
# However : very weak interaction : we lose to much information
"""Version 1"""
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev,0) # average of the xprev

In [27]:
"""Version 2 : Using Matrix multiplication to get the mean"""
# MeanTensor = (line°normalized_TriangularInf @ X) 
W = torch.tril(torch.ones(T,T))
W = W / W.sum(1,keepdim = True) # W is normalized line by line
xbow2  = W @ x
torch.allclose(xbow, xbow2, rtol=1e-4, atol=1e-6)

True

In [29]:
"""Version 3: Softmax"""
tril = torch.tril(torch.ones(T,T))
W = torch.zeros((T,T))
W = W.masked_fill(tril ==0, float('-inf'))
W = F.softmax(W,dim=-1)
xbow3 = W @ x
torch.allclose(xbow,xbow3,rtol=1e-4, atol=1e-6 )
# Only the average of the past and the current token

True

In [8]:
"""Version 4: self-attention"""
B,T,C = 4,8,32 # batch time channels
x = torch.randn(B,T,C)

# single head performing self attention
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False) # y=xWT+b, x = input, W = weight, b=O bc bias = False
value = nn.Linear(C, head_size, bias = False)
k = key(x)   # size = (B,T,16)
q = query(x) # size = (B,T,16)
W = q @ k.transpose(-2,-1) # (B,T,16) * (B,16,T) = (B,T,T)

tril = torch.tril(torch.ones(T,T))
W = W.masked_fill(tril ==0, float('-inf'))
W = F.softmax(W,dim=-1)
v = value(x)

out = W @ v
#out = W @ x

### Feedforward in Transformers

A **feedforward network (FFN)** is applied to each token **individually** after self-attention.  
It typically consists of two linear layers with a **non-linear activation** (like ReLU or GELU) in between:

$$
\text{FFN}(x_i) = W_2 \, \text{ReLU}(W_1 x_i + b_1) + b_2
$$

- **Purpose:** Transform each token's embedding independently to capture more complex patterns.  
- **Non-linearity:** Allows the network to model complex relationships that cannot be represented by linear transformations alone.  

💡 Intuition:  
- Self-attention mixes information **between tokens**.  
- Feedforward mixes information **within each token** in a non-linear way.


### Finally, What is a Transformer ?

**Transformer** = sequence model based on attention mechanisms: composed of **stacked identical blocks** called Transformer blocks.  

Each block typically contains:  
1. **Self-Attention (or Multi-Head Attention)** – lets tokens exchange information within the sequence.  
2. **Feedforward network (FFN)** – transforms each token individually with non-linearities.  
3. **Residual connections + LayerNorm** – stabilize and improve training.  

- Multiple blocks are **stacked**, each with its own learned weights.  
- Each block refines the token embeddings, producing rich, context-aware representations.  